# Quantitative Evaluations of Topic Models
This notebook is for evaluating the quality of the topic models. Mainly it will be a running benchmark of my models + a comparison to LDA (when I set that up)

In [35]:
import pickle
from bertopic import BERTopic
from pathlib import Path
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary

In [17]:
def read_pickle(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

def pickle_object(obj, file_path):
    with open(file_path, "wb") as f:
        pickle.dump(obj, f)

def flatten_list(lst):
    return [elem for sublist in lst for elem in sublist]
    
def get_paragraphs(paragraph_dict):
    return flatten_list(list(paragraph_dict.values()))

In [19]:
# Loading topic model
MODEL_PATH = Path("../models/")
DATA_DIR = Path("../../BscThesisData/data")
topic_model = BERTopic.load(str(MODEL_PATH / "topic_model"), embedding_model="Maltehb/-l-ctra-danish-electra-small-cased")

Some weights of the model checkpoint at C:\Users\jhr/.cache\torch\sentence_transformers\Maltehb_-l-ctra-danish-electra-small-cased were not used when initializing ElectraModel: ['generator.encoder.layer.7.attention.output.LayerNorm.bias', 'generator.encoder.layer.8.attention.self.value.weight', 'generator.encoder.layer.7.attention.output.dense.bias', 'generator.encoder.layer.9.attention.self.key.bias', 'generator.encoder.layer.1.intermediate.dense.bias', 'generator.encoder.layer.5.attention.self.query.weight', 'generator.encoder.layer.3.output.dense.bias', 'generator.encoder.layer.10.attention.output.dense.bias', 'generator.encoder.layer.9.intermediate.dense.bias', 'generator.encoder.layer.2.output.LayerNorm.bias', 'generator.encoder.layer.4.attention.self.value.bias', 'generator.encoder.layer.9.attention.self.query.bias', 'generator.encoder.layer.2.attention.self.key.bias', 'discriminator_predictions.LayerNorm.bias', 'generator.encoder.layer.11.attention.output.LayerNorm.bias', 'gener

In [71]:
clean_paragraphs = read_pickle(DATA_DIR / "paragraph_dict.pkl")
paragraph_list = get_paragraphs(clean_paragraphs)
vectorizer = read_pickle(MODEL_PATH / "vectorizer.pkl")
X = vectorizer.fit_transform(paragraph_list)

In [41]:
# transform sparse matrix into gensim corpus
from gensim.matutils import Sparse2Corpus
corpus_vect_gensim = Sparse2Corpus(X, documents_columns=False)
dictionary = Dictionary.from_corpus(corpus_vect_gensim,
                                    id2word=dict((id, word) for word, id in vectorizer.vocabulary_.items()))

In [63]:
tokenizer = vectorizer.build_tokenizer()
tokenized_docs = [[word for word in tokenizer(doc) if not word.lower() in vectorizer.stop_words] for doc in paragraph_list]

In [103]:
# Getting the Topics 
topics = topic_model.get_topics()
topic_list = [[item[0] for item in topic] for topic in topics.values()]
assert len(topic_list) == len(list(topics.keys()))
assert type(topic_list) == list
assert type(topic_list[0]) == list
assert type(topic_list[0][0]) == str

In [102]:
cm = CoherenceModel(topics=topic_list, coherence="c_v", dictionary=dictionary, texts=tokenized_docs)

KeyError: 'skatterådet'

In [90]:
cm.get_coherence_per_topic()

C:\Users\jhr\Anaconda3\envs\bertopic_explore\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
C:\Users\jhr\Anaconda3\envs\bertopic_explore\lib\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


[nan, nan, 0.484579587062181, 0.6732075346397589, nan, nan]

In [106]:
all_words = set(word for doc in tokenized_docs for word in doc)
topic_words = set(topic_list[0])

"danmark" in topic_words 
"danmark" in all_words

False

In [108]:
# Test if it works to remove invalid words :)) 
clean_topic_list = [[word for word in topic if word in all_words] for topic in topic_list]
cm = CoherenceModel(topics=clean_topic_list, coherence="c_v", dictionary=dictionary, texts=tokenized_docs)
cm.get_coherence_per_topic()

[0.3043144840452862,
 0.27015012488366497,
 0.4865898101777429,
 0.6728787468629828,
 0.41105431905355916,
 0.416226571950254]

It does!

In [70]:
# Minimum reproducible example of nan
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
'This is the first document.',
'This document is the second document.',
'And this is the third one.',
'Is this the first document?',]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)



[['Sagen',
  'angik',
  'skønsmæssig',
  'forhøjelse',
  'sagsøgerens',
  'skattepligtige',
  'indkomst',
  'indkomstårene',
  '2013',
  '2014',
  'skattemyndighederne',
  'foretaget',
  'grundlag',
  'privatforbrugsopgørelse',
  'sagsøgerens',
  'husstand',
  'udviste',
  'negativt',
  'privatforbrug',
  'husstanden',
  'pågældende',
  'indkomstår'],
 ['Retten',
  'fandt',
  'godtgjort',
  'sagsøgeren',
  'årene',
  '2000',
  '2012',
  'sparet',
  '600',
  '000',
  '700',
  '000',
  'kr',
  'kontanter',
  'brugt',
  'finansiere',
  'privatforbrug',
  '2013',
  '2014'],
 ['Retten',
  'fandt',
  'godtgjort',
  'SKATs',
  'skønsmæssige',
  'ansættelse',
  'husstandens',
  'privatforbrug',
  'baseret',
  'dels',
  'faktiske',
  'konstaterede',
  'udgifter',
  'dels',
  'skøn',
  'fastsat',
  'højt'],
 ['Endelig',
  'fandt',
  'retten',
  'godtgjort',
  'del',
  'skønsmæssige',
  'forhøjelse',
  'foretaget',
  'baggrund',
  'opgørelse',
  'husstandens',
  'privatforbrug',
  'henføres',
  '